In [47]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
from typing import List
from flair.data import Token
import sys,os
import tqdm
import pycld2 as cld2
#from Levenshtein import distance 
from datetime import datetime, timedelta
sys.path.append('../')


In [48]:
import configparser
configParser = configparser.RawConfigParser()   
configFilePath = r'config.txt'
configParser.read(configFilePath)
csvpath = configParser.get('paths', 'csvpath')
outpath = configParser.get('paths', 'outpath')
outpath,csvpath

('./', './data')

In [49]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [50]:
def remove_eng(data):
    return ' '.join(re.sub(u"[^ሀ-፼]", " ", data).split()).strip()
     

In [51]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [52]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

# get today's tweet 

In [53]:
remove = ['\xa087','\xa0', '|', '/','…','«','"','“','”','»','-',
          '$','%','^','&','*','›','’','‘','‹','<','>','`','´','~','=','+']
outfile = outpath+"todays_news.txt"
os.remove(outfile) if os.path.exists(outfile) else None
allnews = []
for news in glob.glob(csvpath+"/*.csv"):
    allnews.append(news)

with open (outfile,"a", encoding="utf-8") as todays_news:
    lines_seen = set() # holds lines already seen
    for news in tqdm.tqdm(allnews,  position=0, leave=True):
        data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
        print(news)
        break
        todaydate = datetime.today().strftime('%Y-%m-%d') #get the date of today from server in %Y-%m-%d format
        data = data.loc[(data['Date'] >= todaydate)]
        #data[(data['Date'] >= todaydate)]
        data = data[data.Content.duplicated()==False].reset_index()
        data.Content = data.Content.apply(lambda x: remove_eng(x))
        for token in remove:
            data.Content = data.Content.apply(lambda x: x.replace(token,' '))
            #data[(data['Date'] >= datetime.date(datetime.now()))]
        
        for text in data.Content:
            for s in amseg.tokenize_sentence(text):
                isReliable, textBytesFound, details  = cld2.detect(s)
                if details[0][1] =='am' and s not in lines_seen: # check if line is not duplicate:                  
                    todays_news.write(' '.join(t.text for t in amseg.amharic_tokenizer(s))+'\n')
                    lines_seen.add(s)

  0%|          | 0/2 [00:01<?, ?it/s]

./data\20-12-2020-news.csv


In [ ]:
import json
r = requests.get('http://localhost:9200') 
i = 1
if r.status_code == 200:
    for s in allsentence:
        body = {
            "sentence":s
        }
        es.index(index='amcor', doc_type='all', id=i, body=body)
        i=i+1
        if (i%10000 == 0):
            print(i)

In [46]:
file = open("todays_news.txt", "r", encoding="utf-8")
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print(line_count)

0


In [20]:
#df = df.drop(['Unnamed: 0','id','location','time','user'], axis =1)
#data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
#todaytweet=df[(df['date'] >= datetime.today().strftime('%Y-%m-%d'))] #get the date of today from server

# Search by keyword

In [ ]:
with open("all_sentences.txt", "r",encoding="utf-8") as f:
    content = f.read()
list_of_sentences = content.splitlines()
sentences_series = pd.Series(list_of_sentences)
tagret_sentences = sentences_series[sentences_series.str.contains("ምርጫ")] 

# text similarity

In [ ]:
target_news = 'ታዛቢዎች ግን ምርጫዎቹ ነፃ ፍትሃዊ ስለመሆናቸው ይጠራጠራሉ'
news=set()
def match(target_news):
  for new in sentences_series:
    if distance(target_news, new) <=10:
        news.add(new)
match(target_news)       
print (news)